<a href="https://colab.research.google.com/github/MuhammadHelmyOmar/PyTorch_Projects/blob/main/custom_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recap

Steps to solve a machine learning problem:
1. Find a dataset
2. Turn the dataset into numbers
3. Create a model or find a pretrained one to extract the patterns in the data for prediction

# Intro

- Custom Data: a collection of data relating to your problem and can be comprised from almost anything.
